In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\sb075216\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = "bucket4sm"
print("USing bucket "+ bucket + " From "+ region)

USing bucket bucket4sm From us-east-1


In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
df.shape

(2000, 21)

In [6]:
df['price_range'].value_counts(normalize= True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [7]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [8]:
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [9]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [10]:
label = features.pop(-1)
label

'price_range'

In [11]:
x=df[features]
y=df[label]

In [12]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [13]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state=0)

In [16]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [17]:
trainX.to_csv("train-V-1.csv",index=False)
testX.to_csv("test-V-1.csv",index=False)

In [ ]:
sk_prefix = "sagemaker/mob_price_classification/sklearncontainer"
trainpath = sess.upload_data(
    path='train-V-1.csv',bucket=bucket,key_prefix=sk_prefix
)
testpath = sess.upload_data(
    path='test-V-1.csv',bucket=bucket,key_prefix=sk_prefix
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\sb075216\Desktop\Aws_sagemaker                                                          │
│ Project\myenv\Lib\site-packages\botocore\httpsession.py:464 in send                              │
│                                                                                                  │
│   461 │   │   │   │   conn.proxy_headers['host'] = host                                          │
│   462 │   │   │                                                                                  │
│   463 │   │   │   request_target = self._get_request_target(request.url, proxy_url)              │
│ ❱ 464 │   │   │   urllib_response = conn.urlopen(                                                │
│   465 │   │   │   │   method=request.method,                                                     │
│   466 │   │   │   │   url=request_target,                                                        │
│   467 │   │   │   │   body=request.body,                                                         │
│                                                                                                  │
│ c:\Users\sb075216\Desktop\Aws_sagemaker                                                          │
│ Project\myenv\Lib\site-packages\urllib3\connectionpool.py:787 in urlopen                         │
│                                                                                                  │
│    784 │   │   │   response_conn = conn if not release_conn else None                            │
│    785 │   │   │                                                                                 │
│    786 │   │   │   # Make the request on the HTTPConnection object                               │
│ ❱  787 │   │   │   response = self._make_request(                                                │
│    788 │   │   │   │   conn,                                                                     │
│    789 │   │   │   │   method,                                                                   │
│    790 │   │   │   │   url,                                                                      │
│                                                                                                  │
│ c:\Users\sb075216\Desktop\Aws_sagemaker                                                          │
│ Project\myenv\Lib\site-packages\urllib3\connectionpool.py:493 in _make_request                   │
│                                                                                                  │
│    490 │   │   # conn.request() calls http.client.*.request, not the method in                   │
│    491 │   │   # urllib3.request. It also calls makefile (recv) on the socket.                   │
│    492 │   │   try:                                                                              │
│ ❱  493 │   │   │   conn.request(                                                                 │
│    494 │   │   │   │   method,                                                                   │
│    495 │   │   │   │   url,                                                                      │
│    496 │   │   │   │   body=body,                                                                │
│                                                                                                  │
│ c:\Users\sb075216\Desktop\Aws_sagemaker                                                          │
│ Project\myenv\Lib\site-packages\botocore\awsrequest.py:96 in request                             │
│                                                                                                  │
│    93 │   │   else:                                                                              │
│    94 │   │   │   self._expect_header_set = False                                                │
│    95 │   │   │   self.response_class = self._original_resp

In [24]:
print(trainpath)
print(testpath)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(trainpath)                                                                             │
│   2 print(testpath)                                                                              │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainpath' is not defined

In [18]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os
import numpy as np
import pandas as pd
import argparse
import sklearn

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return clf

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Use environment variables with proper fallbacks
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR", "/opt/ml/model"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", "/opt/ml/input/data/train"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST", "/opt/ml/input/data/test"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args = parser.parse_args()

    print("SKLearn Version: ", sklearn.__version__)

    # List directory contents to debug
    print(f"Contents of train directory: {os.listdir(args.train)}")
    print(f"Contents of test directory: {os.listdir(args.test)}")

    # Read training data
    train_path = os.path.join(args.train, args.train_file)
    test_path = os.path.join(args.test, args.test_file)

    print("[INFO] Reading training data")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    print(f"Training data shape: {train_df.shape}")
    print(f"Testing data shape: {test_df.shape}")

    # Extract features and labels
    features = list(train_df.columns)
    label = features.pop(-1)  # Assumes label is last column

    print("Column order:")
    print(features)
    print()

    print("Label column is: ", label)
    print()

    X_train = train_df[features]
    y_train = train_df[label]

    # Extract test features
    # Check if test data has the label column
    has_label_in_test = label in test_df.columns
    
    if has_label_in_test:
        X_test = test_df[features]
        y_test = test_df[label]
    else:
        print(f"Warning: Test data does not contain the label column '{label}'")
        print("Will only generate predictions, no evaluation metrics")
        # Use all columns in test data as features
        X_test = test_df
        y_test = None

    print("[INFO] Training RandomForest Model")
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=0)
    model.fit(X_train, y_train)

    # Create model directory if it doesn't exist
    model_dir = args.model_dir
    os.makedirs(model_dir, exist_ok=True)

    # Save the trained model
    model_path = os.path.join(model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print(f"Model saved to {model_path}")

    # Model evaluation (only if we have test labels)
    if y_test is not None:
        y_pred_test = model.predict(X_test)
        test_acc = accuracy_score(y_test, y_pred_test)
        test_rep = classification_report(y_test, y_pred_test)

        print("\n----METRICS RESULT FOR TESTING DATA----\n")
        print(f"[Testing] Model Accuracy is: {test_acc}")
        print("[Testing] Testing Report: ")
        print(test_rep)
    else:
        # Just generate predictions
        y_pred_test = model.predict(X_test)
        print("\n----PREDICTIONS FOR TESTING DATA----\n")
        print("First 10 predictions:", y_pred_test[:10])
        
        # Optionally save predictions to a file
        pred_df = pd.DataFrame({'predicted_price_range': y_pred_test})
        pred_path = os.path.join(args.model_dir, "predictions.csv")
        pred_df.to_csv(pred_path, index=False)
        print(f"Predictions saved to {pred_path}")


Overwriting script.py


In [19]:
from sagemaker.sklearn.estimator import SKLearn
import sagemaker

# Use a newer framework version
FRAMEWORK_VERSION = "1.0-1"  # Using a more recent version

# Define your input channels
train_input = sagemaker.inputs.TrainingInput(
    s3_data="s3://bucket4sm/sagemaker/mob_price_classification/sklearncontainer", 
    content_type="text/csv"
)

test_input = sagemaker.inputs.TrainingInput(
    s3_data="s3://bucket4sm/sagemaker/mob_price_classification/sklearncontainer",
    content_type="text/csv"
)

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::102641712713:role/Sage_example",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
        # If your files have different names, specify them here
        "train-file": "train-V-1.csv",  
        "test-file": "test-V-1.csv"
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600
)

# Specify the input channels when fitting
sklearn_estimator.fit({
    "train": train_input,
    "test": test_input},
    wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2025-05-14-16-47-22-330


2025-05-14 16:47:31 Starting - Starting the training job...
2025-05-14 16:47:58 Starting - Preparing the instances for training...
2025-05-14 16:48:26 Downloading - Downloading input data...
2025-05-14 16:48:52 Downloading - Downloading the training image...
2025-05-14 16:49:53 Training - Training image download completed. Training in progress..2025-05-14 16:49:57,161 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-05-14 16:49:57,165 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-14 16:49:57,168 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-05-14 16:49:57,185 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-05-14 16:49:57,488 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-14 16:49:57,492 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
20

In [20]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]


2025-05-14 16:50:18 Starting - Preparing the instances for training
2025-05-14 16:50:18 Downloading - Downloading the training image
2025-05-14 16:50:18 Training - Training image download completed. Training in progress.
2025-05-14 16:50:18 Uploading - Uploading generated training model
2025-05-14 16:50:18 Completed - Training job completed


In [21]:
print("Model artifact location "+ artifact)

Model artifact location s3://sagemaker-us-east-1-102641712713/RF-custom-sklearn-2025-05-14-16-47-22-330/output/model.tar.gz


In [ ]:
artifact

's3://sagemaker-us-east-1-102641712713/RF-custom-sklearn-2025-05-13-04-37-55-668/output/model.tar.gz'

In [22]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::102641712713:role/Sage_example",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [23]:
model

In [25]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2025-05-14-16-56-41


INFO:sagemaker:Creating model with name: custom-sklearn-model2025-05-14-16-53-47
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2025-05-14-16-56-41
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2025-05-14-16-56-41


----------!

In [26]:
endpoint_name


'Custom-sklearn-model-2025-05-14-16-56-41'

In [27]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [ ]:
print(predictor.predict(testX[features][0:2].values.tolist()))

[3 0]


In [30]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '2a305ac2-965d-4a3a-a4a3-87a74945df45',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2a305ac2-965d-4a3a-a4a3-87a74945df45',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 14 May 2025 17:16:04 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}